# Clustering Crypto

In [134]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from urllib.request import Request, urlopen
import json

### Fetching Cryptocurrency Data

In [135]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"
json_request = Request(url)
json_response = urlopen(json_request)
data = json_response.read()
json_data = json.loads(data)

In [136]:
# Create a DataFrame 
crypto_df = pd.DataFrame(json_data['Data']).T
crypto_df.head()

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,BlockTime,AssetLaunchDate,MaxSupply,MktCapPenalty,PlatformType,BuiltOn,SmartContractAddress,DecimalPoints,Difficulty,IsUsedInDefi
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,0,0000-00-00,0,0,NaN,NaN,NaN,NaN,NaN,NaN
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,0,2017-07-01,300,0,token,ETH,0xaec98a708810414878c3bcdf46aad31ded4a4557,18,NaN,NaN
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,0,0000-00-00,0,0,NaN,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,0,0000-00-00,0,0,NaN,NaN,NaN,NaN,NaN,NaN
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Alternatively, use the provided csv file:
# file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame

### Data Preprocessing

In [159]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'

new_crypto_df = crypto_df.drop(columns=['Id','Url','ImageUrl','ContentCreatedOn','Name','Symbol','FullName','Description', 'AssetTokenStatus','BlockTime','AssetLaunchDate', 'MktCapPenalty', 'PlatformType', 'BuiltOn', 'SmartContractAddress', 'DecimalPoints', 'Difficulty', 'Taxonomy', 'Sponsored', 'SortOrder', 'Rating','NetHashesPerSecond','BlockNumber','BlockReward','IsUsedInDefi'])


In [160]:
# Keep only cryptocurrencies that are trading
is_trading_df = new_crypto_df[new_crypto_df.IsTrading == True]
is_trading_df.head(10)

,CoinName,Algorithm,ProofType,IsTrading,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,True,0,0
300,300 token,N/A,N/A,True,300,300
365,365Coin,X11,PoW/PoS,True,0,0
404,404Coin,Scrypt,PoW/PoS,True,0,0
611,SixEleven,SHA-256,PoW,True,0,0
808,808,SHA-256,PoW/PoS,True,0,0
888,Octocoin,N/A,PoW,True,0,0
1337,EliteCoin,X13,PoW/PoS,True,0,0
2015,2015 coin,X11,PoW/PoS,True,0,0
BTCD,BitcoinDark,SHA-256,PoW/PoS,True,NaN,NaN


In [164]:
# Keep only cryptocurrencies with a working algorithm
working_algo_df = is_trading_df.dropna(axis=0, subset=["Algorithm"])
working_algo_df.head(10)

,CoinName,Algorithm,ProofType,IsTrading,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,True,0,0
300,300 token,N/A,N/A,True,300,300
365,365Coin,X11,PoW/PoS,True,0,0
404,404Coin,Scrypt,PoW/PoS,True,0,0
611,SixEleven,SHA-256,PoW,True,0,0
808,808,SHA-256,PoW/PoS,True,0,0
888,Octocoin,N/A,PoW,True,0,0
1337,EliteCoin,X13,PoW/PoS,True,0,0
2015,2015 coin,X11,PoW/PoS,True,0,0
BTCD,BitcoinDark,SHA-256,PoW/PoS,True,NaN,NaN


In [166]:
# Remove the "IsTrading" column
newer_crypto_df = working_algo_df.drop("IsTrading", axis=1)
newer_crypto_df.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,0,0
300,300 token,N/A,N/A,300,300
365,365Coin,X11,PoW/PoS,0,0
404,404Coin,Scrypt,PoW/PoS,0,0
611,SixEleven,SHA-256,PoW,0,0
808,808,SHA-256,PoW/PoS,0,0
888,Octocoin,N/A,PoW,0,0
1337,EliteCoin,X13,PoW/PoS,0,0
2015,2015 coin,X11,PoW/PoS,0,0
BTCD,BitcoinDark,SHA-256,PoW/PoS,NaN,NaN


In [167]:
# Remove rows with at least 1 null value
newer_crypto_df.dropna()
newer_crypto_df.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,0,0
300,300 token,N/A,N/A,300,300
365,365Coin,X11,PoW/PoS,0,0
404,404Coin,Scrypt,PoW/PoS,0,0
611,SixEleven,SHA-256,PoW,0,0
808,808,SHA-256,PoW/PoS,0,0
888,Octocoin,N/A,PoW,0,0
1337,EliteCoin,X13,PoW/PoS,0,0
2015,2015 coin,X11,PoW/PoS,0,0
BTCD,BitcoinDark,SHA-256,PoW/PoS,NaN,NaN


In [168]:
# Remove rows with cryptocurrencies having no coins mined
newer_crypto_df.dropna(inplace=True)
newer_crypto_df.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,0,0
300,300 token,N/A,N/A,300,300
365,365Coin,X11,PoW/PoS,0,0
404,404Coin,Scrypt,PoW/PoS,0,0
611,SixEleven,SHA-256,PoW,0,0
808,808,SHA-256,PoW/PoS,0,0
888,Octocoin,N/A,PoW,0,0
1337,EliteCoin,X13,PoW/PoS,0,0
2015,2015 coin,X11,PoW/PoS,0,0
GEO,GeoCoin,N/A,N/A,0,0


In [169]:
# Drop rows where there are 'N/A' text values
newest_crypto_df = newer_crypto_df.dropna(axis=1)

In [170]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df

coin_name_df = pd.DataFrame(newest_crypto_df["CoinName"])
coin_name_df.head()




,CoinName
42,42 Coin
300,300 token
365,365Coin
404,404Coin
611,SixEleven


In [147]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
updated_df = newest_crypto_df.drop(["CoinName"], axis=1)
updated_df.head

<bound method NDFrame.head of               Algorithm ProofType SortOrder Sponsored  \
LOKI  CryptoNight-Heavy       PoW      2838     False   
BTC             SHA-256       PoW         1     False   
BTS             SHA-512       PoS        10     False   
ETH              Ethash       PoW         2     False   

                                               Taxonomy  \
LOKI  {'Access': '', 'FCA': '', 'FINMA': '', 'Indust...   
BTC   {'Access': 'Permissionless', 'FCA': 'Exchange'...   
BTS   {'Access': 'Permissioned', 'FCA': 'Utility', '...   
ETH   {'Access': 'Permissionless', 'FCA': 'Exchange,...   

                                                 Rating TotalCoinsMined  \
LOKI  {'Weiss': {'Rating': '', 'TechnologyAdoptionRa...        52062104   
BTC   {'Weiss': {'Rating': 'A-', 'TechnologyAdoption...        18581718   
BTS   {'Weiss': {'Rating': 'D+', 'TechnologyAdoption...     2.99482e+09   
ETH   {'Weiss': {'Rating': 'A-', 'TechnologyAdoption...     1.13994e+08   

     BlockNu

In [171]:
# Create dummy variables for text features
dummy_vars = pd.get_dummies(updated_df, columns=["Algorithm", "ProofType"])
dummy_vars.head(10)

,SortOrder,Sponsored,Taxonomy,Rating,TotalCoinsMined,BlockNumber,NetHashesPerSecond,BlockReward,IsUsedInDefi,Algorithm_CryptoNight-Heavy,Algorithm_Ethash,Algorithm_SHA-256,Algorithm_SHA-512,ProofType_PoS,ProofType_PoW
LOKI,2838,False,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",52062104,0,0,0,1,1,0,0,0,0,1
BTC,1,False,"{'Access': 'Permissionless', 'FCA': 'Exchange'...","{'Weiss': {'Rating': 'A-', 'TechnologyAdoption...",18581718,663734,1.68533e+11,6.25,0,0,0,1,0,0,1
BTS,10,False,"{'Access': 'Permissioned', 'FCA': 'Utility', '...","{'Weiss': {'Rating': 'D+', 'TechnologyAdoption...",2.99482e+09,54461018,0,0,1,0,0,0,1,1,0
ETH,2,False,"{'Access': 'Permissionless', 'FCA': 'Exchange,...","{'Weiss': {'Rating': 'A-', 'TechnologyAdoption...",1.13994e+08,11540963,310044061945319,3.29321,0,0,1,0,0,0,1


In [146]:
# Standardize data
X_scaled = StandardScaler().fit_transform(dummy_vars)
print(X_scaled[0:5])

TypeError: float() argument must be a string or a number, not 'dict'

### Reducing Dimensions Using PCA

In [95]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)
X_pca

array([[-6.02344401e-01,  1.70150287e-02, -1.21125467e-03],
       [-6.02344401e-01,  1.70150287e-02, -1.21125467e-03],
       [-6.02344401e-01,  1.70150287e-02, -1.21125467e-03],
       ...,
       [ 6.61263657e+00, -8.29948344e-01, -4.22374379e-02],
       [ 6.61263657e+00, -8.29948344e-01, -4.22374379e-02],
       [-6.02344401e-01,  1.70150287e-02, -1.21125467e-03]])

In [ ]:
# Create a DataFrame with the principal components data


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [ ]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values


# Create the Elbow Curve using hvPlot


Running K-Means with `k=<your best value for k here>`

In [ ]:
# Initialize the K-Means model

# Fit the model

# Predict clusters

# Create a new DataFrame including predicted clusters and cryptocurrencies features


### Visualizing Results

#### 3D-Scatter with Clusters

In [ ]:
# Create a 3D-Scatter with the PCA data and the clusters


#### Table of Tradable Cryptocurrencies

In [ ]:
# Table with tradable cryptos


In [ ]:
# Print the total number of tradable cryptocurrencies


#### Scatter Plot with Tradable Cryptocurrencies

In [ ]:
# Scale data to create the scatter plot


In [ ]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
